In [ ]:
import pandas as pd
import numpy as np
import requests
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

# Input 
### PostgreSQL
Henter bilernes registreringsnummer fra flådestyrings tabel

In [ ]:
user = 'xx'
pw = 'xx'
port = 5432
host = 'xx'
db = 'xx'

In [ ]:
con = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

In [ ]:
query = """
    SELECT bilreg 
    FROM lora_flaadestyring.bil_bilreg_euid 
    WHERE bilreg <> '*mangler*'
    """

insert = """
    SELECT a.bilreg
    FROM lora_flaadestyring.bil_bilreg_euid a
    LEFT JOIN lora_flaadestyring.biler b
    ON trim(upper(a.bilreg)) = trim(upper(b."LicensePlateNumber"))
    WHERE b."LicensePlateNumber" is null and a.bilreg <> '*mangler*'
"""

delete = """
    SELECT b."LicensePlateNumber"
    FROM lora_flaadestyring.bil_bilreg_euid a
    RIGHT JOIN lora_flaadestyring.biler b
    ON trim(upper(a.bilreg)) = trim(upper(b."LicensePlateNumber"))
    WHERE a.bilreg is null
"""

In [ ]:
bilreg = pd.read_sql_query(insert,con=con)

In [ ]:
plader = bilreg.bilreg.values

In [ ]:
plader

# Berig
### Motorregister API
Henter biloplysninger fra https://motorregisterapi.com/

In [ ]:
auth = ''
url = 'https://motorregisterapi.com/api/vehicles/get/'

In [ ]:
data = []
for plade in plader:
    url = 'https://motorregisterapi.com/api/vehicles/get/{}'.format(plade)
    r = requests.get(url, auth=(auth, auth))
    d = r.json()
    data.append(d)

In [ ]:
df = pd.DataFrame(data)

Fjerner rækker hvor der ikke blev fundet oplysninger på nummerpladen 

In [ ]:
df.dropna(axis=0, subset=['LicensePlateNumber'], inplace=True)

In [ ]:
df.set_index('LicensePlateNumber', inplace=True)

In [ ]:
df

# Output
Gemmer i PostgreSQL database

In [ ]:
table = 'biler'
schema = 'lora_flaadestyring'

In [ ]:
df.to_sql(table, con, schema=schema, if_exists='replace')

# Gammelt

### T Hansen API

In [ ]:
token = '18506180474'
plader = 'ZM37183 BT60163 BS15899 BL55280 BR93131 BV14903 BS15898 BV14900 BV14901 BZ55107 BZ55115 BZ55106'.split(' ')

In [ ]:
data = []

In [ ]:
for plade in plader:
    url = 'https://dmr.thg.dk/importoer/api/Koeretoejekst/GetTekniskData?token={}&plade={}'.format(token, plade)
    r = requests.get(url)
    d = r.json()
    data.append(d)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.set_index('RegNr', inplace=True)

# SQLALCHEMY

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2

In [ ]:
user = 'xx'
pw = 'xx'
port = 5432
host = 'xxx'
db = 'xx'

In [ ]:
engine = create_engine('postgresql://{0}:{1}@{2}:{3}/{4}'.format(user, pw, host, port, db))

In [ ]:
insert = engine.execute(insert)

In [ ]:
for row in insert:
   print(row['bilreg'])

In [ ]:
query = """
    SELECT bilreg 
    FROM lora_flaadestyring.bil_bilreg_euid 
    WHERE bilreg <> '*mangler*'
"""

insert = """
    SELECT a.bilreg
    FROM lora_flaadestyring.bil_bilreg_euid a
    LEFT JOIN lora_flaadestyring.biler b
    ON trim(upper(a.bilreg)) = trim(upper(b."LicensePlateNumber"))
    WHERE b."LicensePlateNumber" is null and a.bilreg <> '*mangler*'
"""

delete = """
    SELECT b."LicensePlateNumber"
    FROM lora_flaadestyring.bil_bilreg_euid a
    RIGHT JOIN lora_flaadestyring.biler b
    ON trim(upper(a.bilreg)) = trim(upper(b."LicensePlateNumber"))
    WHERE a.bilreg is null
"""